# 프로젝트: 멋진 작사가 만들기

## Step 1. 데이터 다운로드

### https://aiffelstaticprd.blob.core.windows.net/media/documents/song_lyrics.zip
### Song Lyrice 데이터를 다운 받는다.

## Step 2. 데이터 읽어오기

### glob 모듈을 사용하면 파일을 읽어오는 작업을 하기가 아주 용이해요. glob 를 활용하여 모든 txt 파일을 읽어온 후, raw_corpus 리스트에 문장 단위로 저장하도록 할게요!

In [1]:
import re
import numpy as np         
import tensorflow as tf
import glob
import os

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 [' There must be some kind of way outta here', 'Said the joker to the thief', "There's too much confusion"]


## Step 3. 데이터 정제

### 앞서 배운 테크닉들을 활용해 문장 생성에 적합한 모양새로 데이터를 정제하세요!

### preprocess_sentence() 함수를 만든 것을 기억하시죠? 이를 활용해 데이터를 정제하도록 하겠습니다.

In [2]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue   # 길이가 0인 문장은 건너뜁니다.
    if sentence[-1] == ":": continue  # 문장의 끝이 : 인 문장은 건너뜁니다.

    if idx > 9: break   # 일단 문장 10개만 확인해 볼 겁니다.
        
    print(sentence)

 There must be some kind of way outta here
Said the joker to the thief
There's too much confusion
I can't get no relief Business men, they drink my wine
Plowman dig my earth
None were level on the mind
Nobody up at his word
Hey, hey No reason to get excited
The thief he kindly spoke
There are many here among us


In [3]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()       # 소문자로 바꾸고 양쪽 공백을 삭제
  
    # 아래 3단계를 거쳐 sentence는 스페이스 1개를 delimeter로 하는 소문자 단어 시퀀스로 바뀝니다.
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)        # 패턴의 특수문자를 만나면 특수문자 양쪽에 공백을 추가
    sentence = re.sub(r'[" "]+', " ", sentence)                  # 공백 패턴을 만나면 스페이스 1개로 치환
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)  # a-zA-Z?.!,¿ 패턴을 제외한 모든 문자(공백문자까지도)를 스페이스 1개로 치환
    
    sentence = sentence.strip()

    sentence = '<start> ' + sentence + ' <end>'      # 이전 스텝에서 본 것처럼 문장 앞뒤로 <start>와 <end>를 단어처럼 붙여 줍니다
    
    return sentence

print(preprocess_sentence("This @_is ;;;sample        sentence."))   # 이 문장이 어떻게 필터링되는지 확인해 보세요.

<start> this is sample sentence . <end>


In [4]:
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
        
    corpus.append(preprocess_sentence(sentence))
        
corpus[:10]

['<start> there must be some kind of way outta here <end>',
 '<start> said the joker to the thief <end>',
 '<start> there s too much confusion <end>',
 '<start> i can t get no relief business men , they drink my wine <end>',
 '<start> plowman dig my earth <end>',
 '<start> none were level on the mind <end>',
 '<start> nobody up at his word <end>',
 '<start> hey , hey no reason to get excited <end>',
 '<start> the thief he kindly spoke <end>',
 '<start> there are many here among us <end>']

### 추가로 지나치게 긴 문장은 다른 데이터들이 과도한 Padding을 갖게 하므로 제거합니다. 너무 긴 문장은 노래가사 작사하기에 어울리지 않을수도 있겠죠.

### 그래서 이번에는 문장을 토큰화 했을 때 토큰의 개수가 15개를 넘어가는 문장을 학습데이터에서 제외하기를 권합니다.

## Step 4. 평가 데이터셋 분리

### 훈련 데이터와 평가 데이터를 분리하세요!

### tokenize() 함수로 데이터를 Tensor로 변환한 후, sklearn 모듈의 train_test_split() 함수를 사용해 훈련 데이터와 평가 데이터를 분리하도록 하겠습니다. 단어장의 크기는 12,000 이상으로 설정하세요! 총 데이터의 20%를 평가 데이터셋으로 사용해 주세요!

In [5]:
def tokenize(corpus):
    # 텐서플로우에서 제공하는 Tokenizer 패키지를 생성
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=7000,  # 전체 단어의 개수 
        filters=' ',    # 별도로 전처리 로직을 추가할 수 있습니다. 이번에는 사용하지 않겠습니다.
        oov_token="<unk>"  # out-of-vocabulary, 사전에 없었던 단어는 어떤 토큰으로 대체할지
    )
    tokenizer.fit_on_texts(corpus)   # 우리가 구축한 corpus로부터 Tokenizer가 사전을 자동구축하게 됩니다.

    # 이후 tokenizer를 활용하여 모델에 입력할 데이터셋을 구축하게 됩니다.
    tensor = tokenizer.texts_to_sequences(corpus)   # tokenizer는 구축한 사전으로부터 corpus를 해석해 Tensor로 변환합니다.

    # 입력 데이터의 시퀀스 길이를 일정하게 맞추기 위한 padding  메소드를 제공합니다.
    # maxlen의 디폴트값은 None입니다. 이 경우 corpus의 가장 긴 문장을 기준으로 시퀀스 길이가 맞춰집니다.
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen=16)  
# a = [] 
# for i in tensor:
    #if > 16:
    # return(tensor)
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[  2  65 280 ...   0   0   0]
 [  2 107   6 ...   0   0   0]
 [  2  65  16 ...   0   0   0]
 ...
 [  2 238   1 ...   0   0   0]
 [  2  10 502 ...   0   0   0]
 [  2 129  21 ...   0   0   0]] <keras_preprocessing.text.Tokenizer object at 0x7ff20a4a3190>


In [ ]:
# 토큰개수를 줄여볼려고 for문을 쓸려다가 실패 하고 그냥 maxlen으로 줄여보았다.

In [6]:
print(tensor.shape) # 모양 보기 

(175749, 16)


In [7]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10:break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to


In [8]:
src_input = tensor[:, :-1]  # tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다. 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
tgt_input = tensor[:, 1:]    # tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
#scr =x #tgt =y
print(src_input[0])
print(tgt_input[0])

[  2  65 280  27  99 528  19  85 778  93   3   0   0   0   0]
[ 65 280  27  99 528  19  85 778  93   3   0   0   0   0   0]


In [9]:
from sklearn.model_selection import train_test_split

sr = 12000
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, 
                                                          tgt_input,       
                                                          test_size= 0.2,
                                                          shuffle=True)
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)
print("Source Train:", enc_val.shape)
print("Target Train:", dec_val.shape)



Source Train: (140599, 15)
Target Train: (140599, 15)
Source Train: (35150, 15)
Target Train: (35150, 15)


In [ ]:
# 벨류 모양이 궁금해서 한번 보았다. sklearn 모델을 사용할려고 하니 기억을 더듬으면서
# 전에 노드를 돌아보며 train, val 를 나누었다. 

In [10]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1    # tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개

dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
datasetval = tf.data.Dataset.from_tensor_slices((enc_val, dec_val)).shuffle(BUFFER_SIZE)
datasetval = datasetval.batch(BATCH_SIZE, drop_remainder=True)

print(dataset, datasetval)

<BatchDataset shapes: ((256, 15), (256, 15)), types: (tf.int32, tf.int32)> <BatchDataset shapes: ((256, 15), (256, 15)), types: (tf.int32, tf.int32)>


In [ ]:
# val_loss 값을 받기위해서 datasetval 추가 하였다.

## Step 5. 인공지능 만들기

## 모델의 Embedding Size와 Hidden Size를 조절하며 10 Epoch 안에 val_loss 값을 2.2 수준으로 줄일 수 있는 모델을 설계하세요! (Loss는 아래 제시된 Loss 함수를 그대로 사용!)

### 그리고 멋진 모델이 생성한 가사 한 줄을 제출하시길 바랍니다!

In [35]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 512
hidden_size = 2050
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [36]:
for src_sample, tgt_sample in dataset.take(1): break
model(src_sample)

<tf.Tensor: shape=(256, 15, 7001), dtype=float32, numpy=
array([[[ 2.37124143e-04, -1.62596174e-04, -2.73255719e-04, ...,
         -3.84996587e-04,  2.16534769e-04,  1.32960340e-04],
        [ 1.18104438e-03, -3.69949004e-04, -2.70370976e-04, ...,
         -4.73939086e-04,  3.91216046e-04, -1.57012910e-04],
        [ 1.86183874e-03, -6.89579814e-04, -2.07470424e-04, ...,
         -4.42810095e-04,  5.05483767e-04, -5.12034632e-04],
        ...,
        [-1.71579048e-03, -1.93317293e-03,  6.37628196e-04, ...,
          1.74744506e-04,  1.73397973e-04, -1.88075611e-03],
        [-1.48868409e-03, -1.65307370e-03,  7.63345684e-04, ...,
         -4.00342047e-04, -3.27407179e-04, -2.47077737e-03],
        [-1.42174447e-03, -1.69122184e-03,  7.06545019e-04, ...,
         -2.66473857e-04, -6.72848953e-04, -3.22716194e-03]],

       [[-3.33154923e-04,  9.53148701e-05, -1.62154814e-04, ...,
         -1.95332068e-05,  3.91133333e-04, -3.64100706e-04],
        [-3.76414682e-04,  6.01103064e-04, -1.

In [37]:
model.summary()

Model: "text_generator_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      multiple                  3584512   
_________________________________________________________________
lstm_6 (LSTM)                multiple                  21016600  
_________________________________________________________________
lstm_7 (LSTM)                multiple                  33628200  
_________________________________________________________________
dense_3 (Dense)              multiple                  14359051  
Total params: 72,588,363
Trainable params: 72,588,363
Non-trainable params: 0
_________________________________________________________________


In [38]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, validation_data = datasetval,epochs=10)

Epoch 1/10
549/549 [==============================] - 193s 352ms/step - loss: 3.1993 - val_loss: 2.8130
Epoch 2/10
549/549 [==============================] - 193s 352ms/step - loss: 2.6504 - val_loss: 2.5594
Epoch 3/10
549/549 [==============================] - 193s 352ms/step - loss: 2.3604 - val_loss: 2.3894
Epoch 4/10
549/549 [==============================] - 193s 352ms/step - loss: 2.0762 - val_loss: 2.2594
Epoch 5/10
549/549 [==============================] - 193s 352ms/step - loss: 1.7893 - val_loss: 2.1588
Epoch 6/10
549/549 [==============================] - 193s 352ms/step - loss: 1.5178 - val_loss: 2.0933
Epoch 7/10
549/549 [==============================] - 193s 352ms/step - loss: 1.2825 - val_loss: 2.0632
Epoch 8/10
549/549 [==============================] - 193s 352ms/step - loss: 1.1024 - val_loss: 2.0664
Epoch 9/10
549/549 [==============================] - 193s 352ms/step - loss: 0.9848 - val_loss: 2.0834
Epoch 10/10
549/549 [==============================] - 193s 352m

In [33]:
# model.fit 부분에 validation_data 값을 입력 하는 부분에서 노드들을 다시 돌아보고 구글링을 해봐도 도저히 답을 찾을수가 없었다.
# 그리고 마침 들어와있는 해선님에게 도움을 요청하니 TF홈페이지에서 찾아보셨고 저에게 힌트를 줘서 많은 도움이 되었다.
# validation_data = 256 ㅡ>512 변경 
# hidden_size = 1024 ㅡ> 2050 변경 
# 변경전에는 val_loss 2.3xxx 나와 변경후 2.11 값을 얻었다. 

In [34]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환합니다.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성해야 합니다. 
    while True:
        predict = model(test_tensor)  # 입력받은 문장의 텐서를 입력합니다. 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]   # 우리 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 됩니다. 

        # 우리 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙여 줍니다. 
        test_tensor = tf.concat([test_tensor, 
																 tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 우리 모델이 <end>를 예측했거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측해야 합니다.
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환합니다. 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated   # 이것이 최종적으로 모델이 생성한 자연어 문장입니다.

In [24]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you so much <end> '

## 프로젝트를 마치며...

### 이번프로젝트는 문제부분이 dataset val 정의하는거랑 model.fit 에서 validation_data 값을 정의 하는부분이 어려웠다. 다행이 datasetval부분은 전 익스 노드를 보면서 상기 시키며 수정 할수있었고 validation_data 값은 조금많이 힘들었지만 팀원도움으로 잘해결할수있었다. 

### 이번에는 토큰을 줄이는 코드를 짜볼려고 이것저것 해보았지만... 이번에는 그냥 넘어가기루 했다.. 나에게는 아직 많은 과제들이 기다리고있으니... ^^ 

### 다음번에는 모든 권고사항이던지 루브릭에 맞추어 더 발전해야겠다. 